In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers.modeling_outputs import BaseModelOutput
import torch
from IPython.display import display, HTML

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
def add_noise_with_snr(encoder_output, target_snr_db):
    """
    Add noise to the encoder output based on a target SNR in dB.
    
    Parameters:
    - encoder_output: torch.Tensor, the encoder's output (last_hidden_state).
    - target_snr_db: float, the desired signal-to-noise ratio in dB.
    
    Returns:
    - noisy_encoder_output: torch.Tensor, encoder output with added noise.
    """
    # Convert SNR from dB to linear scale
    target_snr_linear = 10 ** (target_snr_db / 10)
    
    # Calculate power of the signal
    signal_power = torch.mean(encoder_output ** 2)
    
    # Calculate required noise power for the target SNR
    noise_power = signal_power / target_snr_linear
    noise = torch.randn_like(encoder_output) * torch.sqrt(noise_power)
    
    # Add noise to the encoder output
    noisy_encoder_output = encoder_output + noise
    return noisy_encoder_output


In [3]:
# Define input text and encode
# input_text = "Who is Donald Trump?"
input_text = """
Summarize the text: In telecommunications, orthogonal frequency-division multiplexing (OFDM) 
is a type of digital transmission used in digital modulation for encoding digital (binary) data
on multiple carrier frequencies. OFDM has developed into a popular scheme for wideband digital 
communication, used in applications such as digital television and audio broadcasting, DSL internet 
access, wireless networks, power line networks, and 4G/5G mobile communications.
""".replace("\n", "")
# input_text = "Tranlate from English to Deutsche: How are you?"
# input_text = "Tranlate English to Deutsche: My name is Ziyue and I live in Sweden."
# input_text = "Translate English to Deutsche: Where is the nearest train station?"
# input_text = "Translate English to Deutsche: I would like to order a coffee, please."
# input_text = "Translate English to Deutsche: Can you help me find my hotel?"
# input_text = "Translate English to Deutsche: What time does the museum open?"
# input_text = "Translate English to Deutsche: How much does this ticket cost?"
# input_text = "Translate English to Deutsche: Do you speak English?"
# input_text = "Translate English to Deutsche: My favorite color is blue."
# input_text = "Translate English to Deutsche: I am a student learning German."
# input_text = "Translate English to Deutsche: This is my first time visiting Berlin."
# input_text = "Translate English to Deutsche: I enjoy listening to classical music."
# input_text = "Translate English to Deutsche: I would like to check in to my room."
# input_text = "Translate English to Deutsche: The weather today is sunny and warm."
# input_text = "Translate English to Deutsche: I need to buy a ticket to Munich."
# input_text = "Translate English to Deutsche: Could you recommend a good restaurant nearby?"
# input_text = "Translate English to Deutsche: I will stay here for a week."
# input_text = "Translate Deutsche to English: Wo ist das nächste Restaurant?"
# input_text = "Translate Deutsche to English: Ich bin hier im Urlaub."
# input_text = "Translate Deutsche to English: Können Sie mir bitte helfen?"
# input_text = "Translate Deutsche to English: Ich habe meinen Schlüssel verloren."
# input_text = "Translate Deutsche to English: Die Landschaft hier ist wunderschön."
# input_text = "Translate Deutsche to English: Das Essen hier ist sehr lecker."
# input_text = "Translate Deutsche to English: Wie lange dauert die Fahrt?"
# input_text = "Translate Deutsche to English: Ich möchte gerne ein Zimmer reservieren."
# input_text = "Translate Deutsche to English: Ich komme aus Deutschland."
# input_text = "Translate Deutsche to English: Das ist mein Freund, er wohnt in Hamburg."
# input_text = "Translate Deutsche to English: Welche Sehenswürdigkeiten gibt es hier?"
# input_text = "Translate Deutsche to English: Ich suche einen Supermarkt in der Nähe."
# input_text = "Translate English to French: Can I get directions to the airport?"
# input_text = "Translate English to French: I am traveling with my family."
# input_text = "Translate English to French: I would like to book a table for two."
# input_text = "Translate English to French: How far is the beach from here?"
# input_text = "Translate English to French: I am allergic to peanuts."
# input_text = "Translate English to French: Could I have the bill, please?"
# input_text = "Translate English to French: I am learning French for my studies."
# input_text = "Translate English to French: This is a beautiful place to visit."

In [6]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# Get encoder output without noise
with torch.no_grad():
    encoder_outputs = model.encoder(input_ids=input_ids)

# Generate output without noise
baseline_outputs = model.generate(
    input_ids=None,
    encoder_outputs=encoder_outputs,
    max_length=200,
    do_sample=True,
    temperature=0.5
)
baseline_text = tokenizer.decode(baseline_outputs[0], skip_special_tokens=True)

# Add noise with a target SNR and generate noisy output
target_snr = 1  # Set target SNR
noisy_encoder_output = add_noise_with_snr(encoder_outputs.last_hidden_state, target_snr)
modified_encoder_outputs = BaseModelOutput(last_hidden_state=noisy_encoder_output)

# Generate output with the noisy encoder output
noisy_outputs = model.generate(
    input_ids=None,
    encoder_outputs=modified_encoder_outputs,
    max_length=200,
    do_sample=True,
    temperature=0.5
)
noisy_text = tokenizer.decode(noisy_outputs[0], skip_special_tokens=True)

# Display both outputs
display(HTML(f"<p style='font-size:15px;'>{input_text}</p>"))
print('Without Noise:')
display(HTML(f"<p style='font-size:20px; font-family:\"Comic Sans MS\", cursive;'> {baseline_text}</p>"))
print(f'With Noise (SNR = {target_snr} dB):')
display(HTML(f"<p style='font-size:20px; font-family:\"Comic Sans MS\", cursive;'> {noisy_text}</p>"))

Without Noise:


With Noise (SNR = 1 dB):


In [101]:
# '58' <=> ?
# '0' <=> <start>
# '1' <=> <end>
a = torch.tensor([[58]])
text = tokenizer.decode(a[0], skip_special_tokens=True)
text

'?'

In [89]:
baseline_outputs[0]

tensor([    0, 17913, 30979,  7666,     1], device='cuda:0')

In [90]:
noisy_outputs

tensor([[ 0,  3, 76, 89, 32,  3, 89,  9,  3,  7, 17,  3, 63,  3,  7,  3, 17,  3,
          7,  3, 17,  3,  7,  3, 17,  3,  7,  3, 17,  3,  7,  3, 17,  3,  7,  3,
         17,  3,  7,  3, 17,  3,  7,  3, 17,  3,  7,  3, 17,  3,  7,  3, 17,  3,
          7,  3, 17,  3,  7,  3, 17,  3,  7,  3, 17,  3,  7,  3, 17,  3,  7,  3,
         17,  3,  7,  3, 17,  3,  7,  3, 17,  3,  7,  3, 17,  3,  7,  3, 17,  3,
          7,  3, 17,  3,  7,  3, 17,  3,  7,  3]], device='cuda:0')